# Stock AI Predictor - Parameter Testing on Google Colab

This notebook runs parameter testing for stock prediction patterns using Google Colab's high-performance environment. It's designed to handle computationally intensive parameter combinations, especially for short timeframes like 1-minute data.

## 0. Check Environment and Install Dependencies

First, let's check if we're in Colab and install any needed packages:

In [ ]:
# Verify we're in Colab
import sys

IN_COLAB = 'google.colab' in sys.modules
print(f"Running in Colab: {IN_COLAB}")

if IN_COLAB:
    # Install critical dependencies for pattern mining and data analysis
    print("Installing dependencies...")
    !pip install mplfinance pyclustering matplotlib seaborn pandas numpy scikit-learn tqdm bcrypt sqlitecloud python-dotenv
else:
    print("Not running in Colab - skipping package installation")

## 1. Setup Environment

First, let's mount Google Drive and install required packages:

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone repository if not already in Drive
!git clone https://github.com/YOUR_USERNAME/Stock_AI_Predictor.git /content/Stock_AI_Predictor

# Or use existing repository from Drive
# !ln -s /content/drive/MyDrive/Stock_AI_Predictor /content/Stock_AI_Predictor

In [ ]:
# Verify critical imports are working
def check_import(module_name):
    """Try importing a module and report success/failure"""
    try:
        __import__(module_name)
        return True
    except ImportError as e:
        return False

critical_modules = [
    "mplfinance", 
    "pyclustering", 
    "matplotlib", 
    "numpy", 
    "pandas", 
    "sklearn",
    "tqdm",
    "sqlitecloud"
]

# Check all modules
for module in critical_modules:
    result = "✓ Available" if check_import(module) else "✗ Missing - will install"
    print(f"{module:15}: {result}")

# Install any missing modules
for module in critical_modules:
    if not check_import(module):
        print(f"Installing {module}...")
        !pip install {module} -q

In [ ]:
# Install required packages
!pip install -r /content/Stock_AI_Predictor/requirements.txt
!pip install tqdm bcrypt sqlitecloud python-dotenv mplfinance pyclustering matplotlib seaborn pandas numpy scikit-learn

In [ ]:
# Install Colab-specific requirements if available
colab_req_path = '/content/Stock_AI_Predictor/NoteBook/colab_requirements.txt'

# Create the requirements file if it doesn't exist
if not os.path.exists(colab_req_path):
    print("Creating Colab-specific requirements file...")
    %%writefile {colab_req_path}
    # Colab parameter testing requirements
    # Data processing
    pandas>=1.3.0
    numpy>=1.20.0
    # Visualization
    matplotlib>=3.4.0
    seaborn>=0.11.0
    mplfinance>=0.12.7b0
    # Pattern mining
    pyclustering>=0.10.1
    scikit-learn>=1.0.0
    # Database
    sqlitecloud
    bcrypt
    # Utilities
    tqdm
    python-dotenv

# Install the Colab-specific requirements
print("Installing Colab-specific requirements...")
!pip install -r {colab_req_path}

## 2. Upload Database to Colab

### Option 1: Upload from local machine

In [ ]:
from google.colab import files
import os

# Create directory for database
!mkdir -p /content/Stock_AI_Predictor/Data/Storage

# Upload database file
uploaded = files.upload()
for filename in uploaded.keys():
    # Move uploaded file to the correct location
    !mv "{filename}" "/content/Stock_AI_Predictor/Data/Storage/data.db"
    print(f"Uploaded {filename} to /content/Stock_AI_Predictor/Data/Storage/data.db")

### Option 2: Copy from Google Drive

In [ ]:
import shutil

# Specify the source path in Google Drive
drive_db_path = '/content/drive/MyDrive/Stock_AI_Predictor/Data/Storage/data.db'

# Destination path in Colab
colab_db_path = '/content/Stock_AI_Predictor/Data/Storage/data.db'

# Ensure directory exists
os.makedirs(os.path.dirname(colab_db_path), exist_ok=True)

# Copy the file
if os.path.exists(drive_db_path):
    shutil.copy(drive_db_path, colab_db_path)
    print(f"Database copied from Drive to {colab_db_path}")
else:
    print(f"Database not found at {drive_db_path}")

### Option 3: Connect to SQLite Cloud (if available)

In [ ]:
# Create .env file with SQLite Cloud credentials
%%writefile /content/Stock_AI_Predictor/.env
SQLITECLOUD_URL="your_sqlitecloud_connection_string_here"

In [ ]:
# Copy and create the colab_helpers.py file to the Colab environment
%%writefile /content/Stock_AI_Predictor/NoteBook/colab_helpers.py
# Helper functions for Google Colab environment
import os
import sys
import importlib
import shutil
from typing import Optional, Dict, Tuple, List

def setup_colab_environment():
    """
    Set up the Colab environment for Stock AI Predictor parameter testing
    Returns the correct database path and ensures necessary directories exist
    """
    # Base paths
    project_root = '/content/Stock_AI_Predictor'
    drive_root = '/content/drive/MyDrive/Stock_AI_Predictor'
    
    # Check if we're using Drive or local storage
    using_drive = os.path.exists('/content/drive/MyDrive')
    
    if using_drive:
        print("Using Google Drive for storage")
        # Make sure Drive directories exist
        os.makedirs(f"{drive_root}/Data/Storage", exist_ok=True)
        os.makedirs(f"{drive_root}/Images/ParamTesting", exist_ok=True)
        os.makedirs(f"{drive_root}/docs", exist_ok=True)
        
        # Create the local directories too
        os.makedirs(f"{project_root}/Data/Storage", exist_ok=True)
        
        # Check if DB exists in Drive
        drive_db_path = f"{drive_root}/Data/Storage/data.db"
        local_db_path = f"{project_root}/Data/Storage/data.db"
        
        if os.path.exists(drive_db_path):
            print(f"Found database in Drive: {drive_db_path}")
            # Copy to local if needed
            if not os.path.exists(local_db_path):
                print(f"Copying database to local storage...")
                shutil.copy(drive_db_path, local_db_path)
            return local_db_path
        else:
            print(f"No database found in Drive. Will use local path: {local_db_path}")
            return local_db_path
    else:
        print("Google Drive not mounted. Using local storage only.")
        # Ensure local directory exists
        os.makedirs(f"{project_root}/Data/Storage", exist_ok=True)
        return f"{project_root}/Data/Storage/data.db"

def check_critical_imports():
    """
    Check all critical imports needed for parameter testing
    Returns True if all imports succeed, False otherwise
    """
    critical_modules = [
        "mplfinance", 
        "pyclustering", 
        "matplotlib", 
        "numpy", 
        "pandas", 
        "sklearn",
        "tqdm",
        "sqlitecloud"
    ]
    
    missing_modules = []
    for module in critical_modules:
        try:
            __import__(module)
            print(f"✓ {module:15} available")
        except ImportError:
            print(f"✗ {module:15} missing")
            missing_modules.append(module)
    
    # Install missing modules
    for module in missing_modules:
        print(f"Installing {module}...")
        os.system(f"pip install {module} -q")
    
    return len(missing_modules) == 0

def check_pattern_miner_imports():
    """
    Check imports specifically for Pattern_Miner
    """
    pip_pattern_file = '/content/Stock_AI_Predictor/Pattern/pip_pattern_miner.py'
    
    if not os.path.exists(pip_pattern_file):
        print("Could not find pip_pattern_miner.py at expected location!")
        os.system("find /content/Stock_AI_Predictor -name 'pip_pattern_miner.py'")
        return False
        
    print("Checking pip_pattern_miner.py imports...")
    os.system(f"head -10 {pip_pattern_file}")
    
    # Try importing modules used by pip_pattern_miner
    modules_to_check = [
        "mplfinance", 
        "pyclustering.cluster.silhouette", 
        "pyclustering.cluster.kmeans",
        "pyclustering.cluster.center_initializer",
        "sklearn.preprocessing"
    ]
    
    success = True
    for module in modules_to_check:
        try:
            if "." in module:
                parent = module.split(".")[0]
                importlib.import_module(parent)
            else:
                importlib.import_module(module)
            print(f"✓ Successfully imported {module}")
        except ImportError as e:
            print(f"✗ Failed to import {module}: {e}")
            # Try to install the module
            simple_name = module.split(".")[0] 
            print(f"  Installing {simple_name}...")
            os.system(f"pip install {simple_name} -q")
            success = False
            
    return success

def setup_python_path():
    """Ensure the project root is in the Python path"""
    project_root = '/content/Stock_AI_Predictor'
    
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
        print(f"Added {project_root} to Python path")
    
    # Print the Python path to confirm
    print("Python path includes:")
    for p in sys.path:
        print(f"  {p}")
    
    return project_root in sys.path

## 3. Import and Run Parameter Tester

In [ ]:
# Add the project root to sys.path
import sys
import os

# Ensure the project root is in the path
sys.path.insert(0, '/content/Stock_AI_Predictor')

# Try to import and use the colab helpers if available
try:
    from NoteBook.colab_helpers import setup_colab_environment, check_critical_imports, check_pattern_miner_imports, setup_python_path
    
    # First ensure Python path is set correctly
    setup_python_path()
    
    # Check critical imports are available
    print("\nChecking critical imports:")
    check_critical_imports()
    
    # Check pattern miner imports specifically
    print("\nChecking Pattern Miner imports:")
    check_pattern_miner_imports()
    
    print("Successfully used helper functions")
except ImportError:
    # Fallback to manual setup
    print("Helper functions not available, using manual setup")
    
    # Print the Python path to confirm
    print("Python path includes:")
    for p in sys.path:
        print(f"  {p}")

# Now try importing the parameter tester
print("\nImporting ColabParameterTester:")
try:
    # Try importing the module
    from Experements.ParamTesting.colab_parameter_tester import ColabParameterTester
    print("✓ Successfully imported ColabParameterTester")
except ModuleNotFoundError as e:
    # If the module isn't found, print details and try to fix
    print(f"✗ Import error: {e}")
    module_name = str(e).split("'")[-2]
    print(f"  Installing missing module: {module_name}")
    !pip install {module_name} -q
    
    # Try import again
    try:
        from Experements.ParamTesting.colab_parameter_tester import ColabParameterTester
        print("✓ Successfully imported ColabParameterTester after installing dependencies")
    except Exception as e2:
        print(f"✗ Still having issues: {e2}")
        
        # Try printing the import error in more detail
        import traceback
        traceback.print_exc()
    
    # Try import again
    try:
        from Experements.ParamTesting.colab_parameter_tester import ColabParameterTester
        print("Successfully imported ColabParameterTester after installing dependencies")
    except Exception as e2:
        print(f"Still having issues: {e2}")
        print("Importing pip_pattern_miner directly to check dependencies:")
        !cat /content/Stock_AI_Predictor/Pattern/pip_pattern_miner.py | head -10
except Exception as other_error:
    print(f"Unexpected error: {other_error}")
    print("Checking if pip_pattern_miner.py exists:")
    !ls -la /content/Stock_AI_Predictor/Pattern/

# Use helper function to get correct database path if available
try:
    # Try to use the helper function if available
    db_path = setup_colab_environment()
    print(f"Using database path from helper: {db_path}")
except NameError:
    # Fallback to default path
    db_path = '/content/Stock_AI_Predictor/Data/Storage/data.db'
    print(f"Using default database path: {db_path}")
    
# Create directories if needed
os.makedirs(os.path.dirname(db_path), exist_ok=True)

# Create tester instance with the correct path
try:
    tester = ColabParameterTester(db_path=db_path)
    print("Successfully created ColabParameterTester instance")
except Exception as e:
    print(f"Error creating tester: {e}")
    
    # Try to diagnose the issue
    import traceback
    traceback.print_exc()

In [ ]:
# Check and fix Pattern module imports if needed
import importlib
import os

pip_pattern_file = '/content/Stock_AI_Predictor/Pattern/pip_pattern_miner.py'

if os.path.exists(pip_pattern_file):
    print("Checking pip_pattern_miner.py imports...")
    !head -10 {pip_pattern_file}
    
    # Try importing modules used by pip_pattern_miner
    modules_to_check = [
        "mplfinance", 
        "pyclustering.cluster.silhouette", 
        "pyclustering.cluster.kmeans",
        "pyclustering.cluster.center_initializer",
        "sklearn.preprocessing"
    ]
    
    for module in modules_to_check:
        try:
            if "." in module:
                parent = module.split(".")[0]
                importlib.import_module(parent)
            else:
                importlib.import_module(module)
            print(f"✓ Successfully imported {module}")
        except ImportError as e:
            print(f"✗ Failed to import {module}: {e}")
            # Try to install the module
            simple_name = module.split(".")[0] 
            print(f"  Installing {simple_name}...")
            !pip install {simple_name} -q
else:
    print("Could not find pip_pattern_miner.py at expected location!")
    # Try to locate the file in case path is different
    !find /content/Stock_AI_Predictor -name "pip_pattern_miner.py"

In [ ]:
# Check available stock and timeframe options
stocks = tester.list_stocks()
timeframes = tester.list_timeframes()

print("Available stocks:")
for stock_id, stock_name in stocks:
    print(f"  {stock_id}: {stock_name}")
    
print("\nAvailable timeframes:")
for tf_id, tf_name in timeframes:
    print(f"  {tf_id}: {tf_name}")

In [ ]:
# Run Parameter Tests - Choose One of the Options Below

## Option 1: Run for a specific stock and timeframe with date range
import datetime

# Set date range for testing (optional)
start_date = "2022-01-01"  # Can be None to use all available data
end_date = datetime.datetime.now().strftime("%Y-%m-%d")  # Use today as end date or None

# Run the test and collect the results
results, report = tester.run_parameter_testing_for_stock_timeframe_parallel(
    "GOLD", "M1", 
    start_date=start_date, 
    end_date=end_date
)

## Option 2: Run a quick test (comment out Option 1 if you want to use this)
# results = tester.run_quick_test("GOLD", "M1")

## Option 3: Compare hold period strategies (comment out Options 1 & 2 if you want to use this)
# results = tester.compare_hold_period_strategies("GOLD", "M1")

## Option 4: Run all tests for multiple timeframes (comment out Options 1-3 if you want to use this)
# all_results = tester.run_all_tests(stock_symbol="GOLD")

# Display top 5 parameter combinations if results exist
if results is not None and not isinstance(results, dict):
    print("\nTop 5 parameter combinations:")
    print(results.head(5))
elif results is not None and isinstance(results, dict):
    print("\nResults for multiple timeframes:")
    for tf, res in results.items():
        if not res.empty:
            print(f"\n{tf} - Top 3 combinations:")
            print(res.head(3))

# Check GPU status during execution
!nvidia-smi

## 4. Visualize Results

In [ ]:
# Generate and display visualization
if 'results' in locals() and results is not None and not isinstance(results, dict):
    # For single timeframe results
    fig = tester.visualize_results(results, "GOLD", "M1")
elif 'results' in locals() and results is not None and isinstance(results, dict):
    # For multiple timeframe results
    for tf, res in results.items():
        if not res.empty:
            print(f"\nVisualizing results for {tf}:")
            fig = tester.visualize_results(res, "GOLD", tf)
            
# If all_results exists (from Option 4)
elif 'all_results' in locals() and all_results is not None:
    for tf, res in all_results.items():
        if not res.empty:
            print(f"\nVisualizing results for {tf}:")
            fig = tester.visualize_results(res, "GOLD", tf)

In [ ]:
# Generate optimization report
if 'report' in locals() and report is not None:
    # If we already have a report from previous execution
    print(report)
elif 'results' in locals() and results is not None and not isinstance(results, dict):
    # For single timeframe results
    report = tester.generate_report(results, "GOLD", "M1")
    print(report)
elif 'results' in locals() and results is not None and isinstance(results, dict):
    # For multiple timeframe results
    for tf, res in results.items():
        if not res.empty:
            report = tester.generate_report(res, "GOLD", tf)
            print(f"\nReport for {tf}:")
            print(report)
            
# If all_results exists (from Option 4)
elif 'all_results' in locals() and all_results is not None:
    for tf, res in all_results.items():
        if not res.empty:
            report = tester.generate_report(res, "GOLD", tf)
            print(f"\nReport for {tf}:")
            print(report)

## 5. Save Results Back to Google Drive

In [ ]:
# Copy the updated database back to Google Drive
# This ensures your work is persisted
import shutil
import os

# Source path in Colab
colab_db_path = '/content/Stock_AI_Predictor/Data/Storage/data.db'

# Destination path in Google Drive
drive_db_dir = '/content/drive/MyDrive/Stock_AI_Predictor/Data/Storage'
os.makedirs(drive_db_dir, exist_ok=True)
drive_db_path = os.path.join(drive_db_dir, 'data.db')

# Copy the file
shutil.copy(colab_db_path, drive_db_path)
print(f"Updated database saved to {drive_db_path}")

## 6. Save Report and Visualizations

In [ ]:
# Save report to a file
import os

def save_report(report_content, timeframe):
    """Helper function to save a report to Google Drive"""
    if report_content:
        report_path = f'/content/drive/MyDrive/Stock_AI_Predictor/docs/parameter_optimization_report_{timeframe}.md'
        os.makedirs(os.path.dirname(report_path), exist_ok=True)
        with open(report_path, 'w') as f:
            f.write(report_content)
        print(f"Report saved to {report_path}")

# Save reports based on what data we have
if 'report' in locals() and report is not None:
    # If we have a single report
    save_report(report, "M1")  # Adjust timeframe as needed
    
elif 'results' in locals() and results is not None and isinstance(results, dict):
    # For multiple timeframe results
    for tf, res in results.items():
        if not res.empty:
            tf_report = tester.generate_report(res, "GOLD", tf)
            save_report(tf_report, tf)
            
# If all_results exists (from Option 4)
elif 'all_results' in locals() and all_results is not None:
    for tf, res in all_results.items():
        if not res.empty:
            tf_report = tester.generate_report(res, "GOLD", tf)
            save_report(tf_report, tf)

In [ ]:
# Save visualizations
import matplotlib.pyplot as plt
import os

def save_visualization(stock_symbol, timeframe):
    """Helper function to save a visualization to Google Drive"""
    vis_path = f'/content/drive/MyDrive/Stock_AI_Predictor/Images/ParamTesting/{stock_symbol}_{timeframe}_parameter_test.png'
    os.makedirs(os.path.dirname(vis_path), exist_ok=True)
    
    # Generate visualization again and save
    fig = tester.visualize_results(stock_symbol, timeframe, save_path=vis_path)
    print(f"Visualization saved to {vis_path}")
    return vis_path

# Save visualizations based on what data we have
if 'results' in locals() and results is not None and not isinstance(results, dict):
    # For single timeframe results
    save_visualization("GOLD", "M1")
    
elif 'results' in locals() and results is not None and isinstance(results, dict):
    # For multiple timeframe results
    for tf in results.keys():
        save_visualization("GOLD", tf)
        
# If all_results exists (from Option 4)
elif 'all_results' in locals() and all_results is not None:
    for tf in all_results.keys():
        save_visualization("GOLD", tf)

## 7. Download Database (Optional)

In [ ]:
# Download the updated database to your local machine
from google.colab import files
files.download('/content/Stock_AI_Predictor/Data/Storage/data.db')